# Automated Machine Learning

In [8]:

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
 

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.data.dataset_factory import TabularDatasetFactory

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


In [9]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

AZML
RG-AZML
eastus
a748a023-896b-4719-8f8e-3d1ba62d6e35


In [10]:
# Choose a name for the experiment.
experiment_name = 'ideb-regression'

experiment = Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Run History Name'] = experiment_name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
Subscription ID,a748a023-896b-4719-8f8e-3d1ba62d6e35
Workspace,AZML
Resource Group,RG-AZML
Location,eastus
Run History Name,ideb-regression


## Using AmlCompute

In [11]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "automl-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

### Load Data

In [12]:
datastore = ws.get_default_datastore()
path = "data/Train.csv"
try:
    ideb_dataset = Dataset.get_by_name(ws, name="dataset2019ideb")
except:
    datastore.upload('data', target_path='data')
    # Create TabularDataset & register in workspace
    ideb_dataset = Dataset.Tabular.from_delimited_files([(datastore, path)])
    ideb_dataset = ideb_dataset.register(
        ws, name="ideb_dataset", create_new_version=True,
        description="Dataset for ideb prediction"
    )

## Automl Settings

In [13]:
automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": 'r2_score',
    "enable_early_stopping": True, 
    "experiment_timeout_hours": 0.5, 
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
     "featurization": 'auto',
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(task = 'regression',
                             compute_target = compute_target,
                             training_data = ideb_dataset,
                             label_column_name = "IDEB_2019",
                             **automl_settings
                            )



In [14]:
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on automl-cluster with default configuration
Running on remote compute: automl-cluster
Parent Run ID: AutoML_f7485239-1377-41f3-8bf5-b49e0f342909

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:       

## Result

In [15]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [16]:
remote_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       DONE
DESCRIPTION:  High cardinality features were detected in your inputs and handled.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization
DETAILS:      High cardinality features refer to columns that contain a large percentage of unique values.
+---------------------------------+---------------------------------+
|Column name                      |Column Content Type              |
+=============================

{'runId': 'AutoML_f7485239-1377-41f3-8bf5-b49e0f342909',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-17T01:03:32.512069Z',
 'endTimeUtc': '2021-01-17T01:45:58.744123Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'r2_score',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'automl-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"ideb-regression","subscription_id":"a748a023-896b-4719-8f8e-3d1ba62d6e35","resource_group":"RG-AZML","workspace_name":"AZML","region":"eastus","compute_target":"automl-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"r2_score","task_type":"regression","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_classes":null,"featurization":"auto","_ignore_pack

## Best Model

In [17]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-core, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.2, current version:2.6.1
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0, current version:1.19.0


ModuleNotFoundError: No module named 'azureml.automl.runtime._ml_engine.featurizer_suggestion'

## Metric in best model

In [34]:
best_run_metrics = best_run.get_metrics()
best_run_metrics

{'mean_absolute_error': 0.029838615870874784,
 'mean_absolute_percentage_error': 0.754437290347608,
 'explained_variance': 0.9976823543660368,
 'spearman_correlation': 0.9988073668198089,
 'normalized_mean_absolute_error': 0.004662283729824184,
 'root_mean_squared_log_error': 0.008237186945423953,
 'r2_score': 0.9976804216176918,
 'normalized_root_mean_squared_error': 0.006048348053963425,
 'normalized_median_absolute_error': 0.0040726563190562086,
 'normalized_root_mean_squared_log_error': 0.006191446833612886,
 'root_mean_squared_error': 0.03870942754536592,
 'median_absolute_error': 0.026065000441959734,
 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_ca6e0e8b-0901-4d4f-b9aa-98e07d376f3a_38/residuals',
 'predicted_true': 'aml://artifactId/ExperimentRun/dcid.AutoML_ca6e0e8b-0901-4d4f-b9aa-98e07d376f3a_38/predicted_true'}

## Save the best model

In [35]:
env = best_run.get_environment()
best_run.download_file('outputs/model.pkl', './outputs/automl_ideb.pkl')
best_run.download_file('outputs/scoring_file_v_1_0_0.py', './outputs/score.py')
best_run.download_file('automl_driver.py', './outputs/automl_driver.py')

import joblib  
joblib.dump(fitted_model, "./outputs/model.pkl")

['./outputs/model.pkl']

In [36]:
os.listdir(os.curdir), os.getcwd()

(['.azureml',
  '.config',
  '.ipynb_aml_checkpoints',
  '.ipynb_checkpoints',
  'automl.ipynb',
  'automl.ipynb.amltemp',
  'automl.log',
  'automl2.ipynb',
  'automl2.ipynb.amltemp',
  'automl3.ipynb',
  'automl3.ipynb.amltemp',
  'automl_errors.log',
  'azureml_automl.log',
  'conda_dependencies.yml',
  'config.json',
  'hyperparameter_tuning.ipynb',
  'hyperparameter_tuning.ipynb.amltemp',
  'outputs',
  'score.py.amltemp',
  'train.py',
  'train.py.amltemp',
  'train_explainer.py.amltemp'],
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-d2-v2/code/Users/anderson12')

## Model Deployment

In [37]:
model=best_run.register_model(
                        model_name = 'best_model',
                        model_path = './outputs/',
                        description = "Best model trained with AutoML")
model

Model(workspace=Workspace.create(name='AZML', subscription_id='a748a023-896b-4719-8f8e-3d1ba62d6e35', resource_group='RG-AZML'), name=best_model, id=best_model:18, version=18, tags={}, properties={})

In [39]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
inference_config = InferenceConfig(entry_script='./outputs/score.py')

service = model.deploy(
    workspace=ws,
    name="service-ideb",
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True
)
service.wait_for_deployment(show_output=True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....................................................

In [41]:
service.update(enable_app_insights=True)

## Test the Deploy model

In [ ]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

print(service.scoring_uri)
print(service.swagger_uri)

In [40]:
import requests
import json

# scoring endpoint
scoring_uri = service.scoring_uri


# If the service is authenticated, set the key or token
#key = '<your key or token>'

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
           "Cod_Escola": 4,
           "TAprov2019_Total": 99.9, 
           "TAprov2019_1_serie": 99.2, 
           "TAprov2019_2_serie": 59.1, 
           "TAprov2019_3_serie": 60.5, 
           "TAprov2019_4_serie": 70.5, 
           "Indicador_Rendimento": 0.99, 
           "SAEB2019_Matematica": 365.38, 
           "SAEB2019_Lingua Portuguesa": 351.54, 
           "SAEB2019_Nota Media Padronizada": 7.055853, 
           "IDEB_2019": 7.1
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

MissingSchema: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?

## View the service logs

In [ ]:
print(service.get_logs())

In [ ]:
service.delete()